# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [27]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

The code snippet below outlines the core training procedure for the proposed BERT combined with Transformer model. The trained model checkpoint used for playing the recorded Hangman game is available at the following link: https://drive.google.com/file/d/11cMGIGXGCcZq2pU2mj52jELhQtnSzBqe/view?usp=drive_link

In [ ]:
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from torch.optim import AdamW
from tqdm import tqdm
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# ==== CONFIG ====
MAX_LEN = 12
MAX_ATTEMPTS = 6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== DATASET ====
class HangmanDataset(Dataset):
    def __init__(self, words, tokenizer, augmentations_per_word=14):
        self.samples = []
        self.tokenizer = tokenizer

        for word in words:
            word = word.lower()
            if len(word) > MAX_LEN:
                continue

            unique_chars = list(set(word))

            # Endgame augmentation
            for i in range(2):
                if unique_chars:
                    char = random.choice(unique_chars)
                    masked = ''.join(['_' if c == char else c for c in word])
                    guessed_letters = ''.join(sorted(set(c for c in masked if c != '_')))
                    self.samples.append({"masked": masked, "guessed": guessed_letters, "label": char})

                if len(unique_chars) >= 2:
                    chars = random.sample(unique_chars, 2)
                    masked = ''.join(['_' if c in chars else c for c in word])
                    for char in chars:
                        guessed_letters = ''.join(sorted(set(c for c in masked if c != '_')))
                        self.samples.append({"masked": masked, "guessed": guessed_letters, "label": char})


            for _ in range(augmentations_per_word - len(self.samples) % augmentations_per_word):
                num_to_mask = random.choices([1, 2, 3, 4], weights=[5, 4, 2, 1])[0]
                num_to_mask = min(num_to_mask, len(unique_chars))
                chars = random.sample(unique_chars, num_to_mask)
                masked = ''.join(['_' if c in chars else c for c in word])
                for char in chars:
                    guessed_letters = ''.join(sorted(set(c for c in masked if c != '_')))
                    self.samples.append({"masked": masked, "guessed": guessed_letters, "label": char})

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_text = sample["masked"] + " Guessed: " + sample["guessed"]
        inputs = self.tokenizer(input_text, padding="max_length", truncation=True, max_length=32, return_tensors="pt")
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        label = ord(sample["label"]) - ord('a')
        return input_ids, attention_mask, torch.tensor(label)

# ==== MODEL ====
class BertTransformerClassifier(nn.Module):
    def __init__(self, bert_model="bert-base-uncased", num_layers=2, nhead=8, hidden_dim=256):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.bert.gradient_checkpointing_enable()

        encoder_layer = TransformerEncoderLayer(d_model=self.bert.config.hidden_size, nhead=nhead, dim_feedforward=hidden_dim)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.GELU(),
            nn.LayerNorm(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.LayerNorm(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.LayerNorm(128),
            nn.Dropout(0.1),
            nn.Linear(128, 26)
        )


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state  # (B, T, H)
        x = self.transformer(x.transpose(0, 1)).transpose(0, 1)  # (B, T, H)
        cls_rep = x[:, 0]  # Use CLS token representation
        return self.classifier(cls_rep)

# ==== TRAIN FUNCTION ====
def train_model(model, dataloader, tokenizer, val_words, model_path="bert_transformer_hangman_final.pt", epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        model.bert.train()  # Important with gradient checkpointing
        model.transformer.train()

        running_loss = 0.0
        epoch_loss = 0.0
        print(f"\nEpoch {epoch+1}/{epochs}")
        num_batches = len(dataloader)

        for batch_idx, (input_ids, attention_mask, labels) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
            input_ids, attention_mask, labels = input_ids.to(DEVICE), attention_mask.to(DEVICE), labels.to(DEVICE)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_loss += loss.item()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / num_batches
        print(f"Epoch {epoch+1} complete. Avg Loss: {avg_epoch_loss:.4f}")

        torch.save(model.state_dict(), model_path)
        hangman_evaluate(model, tokenizer, val_words)

# ==== EVALUATION ====
def hangman_evaluate(model, tokenizer, test_words):
    model.eval()
    wins = 0
    for word in test_words:
        word = word.lower()
        masked = ['_' for _ in word]
        guessed_letters = set()
        attempts = 0
        while attempts < MAX_ATTEMPTS and ''.join(masked) != word:
            input_text = ''.join(masked) + " Guessed: " + ''.join(sorted(guessed_letters))
            inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=32)
            input_ids = inputs["input_ids"].to(DEVICE)
            attention_mask = inputs["attention_mask"].to(DEVICE)
            with torch.no_grad():
                logits = model(input_ids, attention_mask)
                probs = torch.softmax(logits, dim=-1)[0]
            for letter in guessed_letters:
                probs[ord(letter) - ord('a')] = 0
            pred_char = chr(torch.argmax(probs).item() + ord('a'))
            guessed_letters.add(pred_char)
            if pred_char in word:
                for i, c in enumerate(word):
                    if c == pred_char:
                        masked[i] = pred_char
            else:
                attempts += 1
        if ''.join(masked) == word:
            wins += 1
    print(f"Validation Win Rate: {wins}/{len(test_words)} = {100 * wins / len(test_words):.2f}%")

# ==== MAIN ====
def main():
    with open("words_250000_train.txt", "r") as f:
        words = [line.strip().lower() for line in f if line.strip().isalpha() and len(line.strip()) <= MAX_LEN]

    val_words = random.sample(words, 1000)
    train_words = list(set(words))

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    dataset = HangmanDataset(train_words, tokenizer)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = BertTransformerClassifier().to(DEVICE)
    train_model(model, dataloader, tokenizer, val_words, epochs=5)

    torch.save(model.state_dict(), "bert_transformer_hangman_final.pt")

    loaded_model = BertTransformerClassifier().to(DEVICE)
    loaded_model.load_state_dict(torch.load("bert_transformer_hangman_final.pt", map_location=DEVICE))

    print("=== Final Evaluation ===")
    hangman_evaluate(loaded_model, tokenizer, val_words)

if __name__ == "__main__":
    main()

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class BertLetterClassifier(nn.Module):
    def __init__(self, bert_model="bert-base-uncased", num_layers=2, nhead=8, hidden_dim=256):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.bert.gradient_checkpointing_enable()

        encoder_layer = TransformerEncoderLayer(d_model=self.bert.config.hidden_size, nhead=nhead, dim_feedforward=hidden_dim)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.GELU(),
            nn.LayerNorm(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.LayerNorm(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.LayerNorm(128),
            nn.Dropout(0.1),
            nn.Linear(128, 26)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state  # (B, T, H)
        x = self.transformer(x.transpose(0, 1)).transpose(0, 1)  # (B, T, H)
        cls_rep = x[:, 0]  # Use CLS token representation
        return self.classifier(cls_rep)
    
    

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
        self.model = BertTransformerClassifier().to(DEVICE)
        self.model.load_state_dict(torch.load("bert_transformer_hangman_final.pt", map_location=DEVICE))
        self.model.eval()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):  # word example: "_ p p _ e "
        with torch.no_grad():
            input_text = ''.join(word.split())  # remove spaces, e.g. "_pp_e"
            # Add guessed letters info if needed: input_text += " Guessed: " + ''.join(sorted(self.guessed_letters))

            inputs = tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=32)
            input_ids = inputs["input_ids"].to(DEVICE)
            attention_mask = inputs["attention_mask"].to(DEVICE)

            logits = self.model(input_ids, attention_mask)
            probs = torch.softmax(logits, dim=-1)[0]

            # Zero out already guessed letters probabilities
            for letter in self.guessed_letters:
                idx = ord(letter) - ord('a')
                probs[idx] = 0

            pred_idx = torch.argmax(probs).item()
            pred_char = chr(pred_idx + ord('a'))
        return pred_char


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [35]:
api = HangmanAPI(access_token="7f837e20b667c281ddfa5140568875", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [37]:
for i in range(1):
  api.start_game(practice=1,verbose=True)
  [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
  print("Succesful games:", total_practice_successes)
  practice_success_rate = total_practice_successes / total_practice_runs
  print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 7dd06572fd07. # of tries remaining: 6. Word: _ _ _ _ _ .
Guessing letter: a
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ _ _ _ '}
Guessing letter: l
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ _ _ _ '}
Guessing letter: o
Sever response: {'game_id': '7dd06572fd07', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ _ o _ _ '}
Guessing letter: s
Sever response: {'game_id': '7dd06572fd07', 'status': 'failed', 'tries_remains': 0, 'word': '_ _ o _ _ '}
Failed game: 7dd06572f

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    success_rate = total_recorded_successes/total_recorded_runs
    print(total_recorded_runs,total_recorded_successes)
    print('overall success rate = %.3f' % success_rate)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [45]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print(total_recorded_runs,total_recorded_successes)
print('overall success rate = %.3f' % success_rate)

1000 588
overall success rate = 0.588
